In [1]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from scraping import init_driver, to_excel

WebDriverException: Message: Can not connect to the Service /Users/sheidamajidi/.wdm/drivers/chromedriver/mac64/124.0.6367.207/chromedriver-mac-arm64/chromedriver


In [ ]:
driver = init_driver()
driver.get("http://www.randhawa.us/games/retailer/nyu.html")


WebDriverException: Message: Can not connect to the Service /Users/sheidamajidi/.wdm/drivers/chromedriver/mac64/124.0.6367.207/chromedriver-mac-arm64/chromedriver


In [ ]:
# extract

driver.find_element(By.ID, "practiceButton").click()
time.sleep(2) 

data = driver.find_element(By.ID, "result-table").text
splitdata = data.split("\n")
columns = splitdata[0].split(" ")
values_split = [row.split(" ") for row in splitdata[1:]]

table = pd.DataFrame(values_split, columns=columns[:4])

#### Exploring strategies

In [16]:
def strategy(table, choices, chosenchoices):
    week = max(table['Week'].astype(int))
    if week > 1 and week - 1 < len(chosenchoices):
        prev_choice = chosenchoices[week-1]
        lift = (int(table['Sales'][week-1]) - int(table['Sales'][week-2])) / int(table['Sales'][week-2])
        lastchoice = max(chosenchoices)
        if lift < 0:
            print("Negative lift:", lift)
            return min(lastchoice+1, 3)
    return 0


In [13]:
attempts = 1000
all_data = []


for attempt in range(attempts):
    print(f"Attempt {attempt+1}/{attempts}")
    chosenchoices = [0]
    driver.find_element(By.ID, "practiceButton").click()
    time.sleep(2) 

    for i in range(15):
        data = driver.find_element(By.ID, "result-table").text
        splitdata = data.split("\n")
        columns = splitdata[0].split(" ")
        values_split = [row.split(" ") for row in splitdata[1:]]

        table = pd.DataFrame(values_split, columns=columns[:4])
        
        if len(table) == 0:
            print("Empty table encountered, breaking the loop.")
            break
        
        choice = strategy(table, [0, 1, 2, 3], chosenchoices)
        chosenchoices.append(choice)
        
        buttons = [
            driver.find_element(By.ID, "maintainButton"),
            driver.find_element(By.ID, "tenButton"),
            driver.find_element(By.ID, "twentyButton"),
            driver.find_element(By.ID, "fortyButton")
        ]
        buttons[choice].click()
        time.sleep(2) 

        if choice == 3:
            data = driver.find_element(By.ID, "result-table").text
            splitdata = data.split("\n")
            columns = splitdata[0].split(" ")
            values_split = [row.split(" ") for row in splitdata[1:]]

            table = pd.DataFrame(values_split, columns=columns[:4])
            chosenchoices.extend([0] * (13 - i))
            break

    foresight = driver.find_element(By.CSS_SELECTOR, "div[style*='position:relative;top:10px;left:6px;']").text
    table['foresight'] = foresight
    table['Choice'] = chosenchoices
    table['diff'] = float(table['foresight'][0].split(": ")[3].replace("%", ""))
    table['revenue'] = float(table['foresight'][0].split(", ")[0].split(": ")[1].replace("$", "").replace(",", ""))
    table['perfect'] = float(table['foresight'][0].split(", ")[1].split(": ")[1].replace("$", "").replace(",", ""))
    
    all_data.append(table)



Attempt 1/1000


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=124.0.6367.203)
Stacktrace:
0   chromedriver                        0x00000001004aa940 chromedriver + 4368704
1   chromedriver                        0x00000001004a2dd4 chromedriver + 4337108
2   chromedriver                        0x00000001000c6c04 chromedriver + 289796
3   chromedriver                        0x00000001000afd00 chromedriver + 195840
4   chromedriver                        0x00000001000afc3c chromedriver + 195644
5   chromedriver                        0x0000000100140d24 chromedriver + 789796
6   chromedriver                        0x00000001000fdab4 chromedriver + 514740
7   chromedriver                        0x00000001000fe50c chromedriver + 517388
8   chromedriver                        0x000000010046ee5c chromedriver + 4124252
9   chromedriver                        0x0000000100473c4c chrome

In [11]:
table

,Week,Price,Sales,Remaining
0,1,60,161,1839
1,2,60,148,1691
2,3,54,228,1463
3,4,54,247,1216
4,5,54,248,968
5,6,54,251,717
6,7,54,262,455
7,8,54,234,221
8,9,48,221,0
9,10,48,0,0
